In [1]:
from api.BinanceData import BinanceData
from utils.CleanData import CleanData
from variables.QuantitativeFunc import QuantitativeFunc
from variables.Target import Target
from variables.TradingIndicators import TradingIndicators
import pandas as pd

In [10]:
# Import de données
BinanceData = BinanceData()
data = BinanceData.load_data("BTCUSDT", days=365)
print(data.head(2))
print(len(data))

       open_time            open            high             low  \
0  1727167500000  63772.01000000  63820.00000000  63757.94000000   
1  1727167800000  63819.17000000  63863.02000000  63800.00000000   

            close       volume     close_time quote_asset_volume  \
0  63819.17000000  62.47638000  1727167799999   3985619.69260590   
1  63861.76000000  77.04959000  1727168099999   4918474.25793110   

   number_of_trades taker_buy_base   taker_buy_quote ignore  
0             12768    33.03768000  2107706.63620400      0  
1             10690    39.27502000  2507016.48080030      0  
105121


In [11]:
# Nettoyage des données 
cleaner = CleanData()
cleaned_data = cleaner.clean_klines_data(data)
print(cleaned_data.index.name)
print(cleaned_data.tail(3))

open_time
                          open       high        low      close    volume  \
open_time                                                                   
2025-09-24 08:35:00  112569.01  112576.03  112494.96  112494.96  27.49263   
2025-09-24 08:40:00  112494.97  112575.99  112494.96  112575.98  19.67526   
2025-09-24 08:45:00  112575.98  112588.00  112575.98  112575.98   8.34664   

                                 close_time  quote_asset_volume  \
open_time                                                         
2025-09-24 08:35:00 2025-09-24 08:39:59.999        3.094417e+06   
2025-09-24 08:40:00 2025-09-24 08:44:59.999        2.214239e+06   
2025-09-24 08:45:00 2025-09-24 08:49:59.999        9.396825e+05   

                     number_of_trades  taker_buy_quote  
open_time                                               
2025-09-24 08:35:00              5883     1.463148e+06  
2025-09-24 08:40:00              3791     1.712315e+06  
2025-09-24 08:45:00              1324   

In [12]:
# Export des données pour travailler sur un ficheir Excel plutôt que de requêter l'API Binance à chaque fois
print(len(cleaned_data))
cleaned_data.to_excel("../data/clean_dataset.xlsx", index=False)

105121


In [3]:
# Import des données depuis le fichier Excel
data = pd.read_excel("../data/clean_dataset.xlsx")

# Création des variables rendement et la cible 
data["return"] = QuantitativeFunc.return_(data['close'])
data["return_10"] = QuantitativeFunc.return_10(data['close'])
data["target"] = Target.compute(data["return"], threshold=0.001)

# ajout des indicateurs techniques
data = TradingIndicators.add_ema(data, price_col="close", window=12, new_col="EMA_12")
data = TradingIndicators.add_macd(data, price_col="close")
data = TradingIndicators.add_bollinger_bands(data, price_col="close")
data = TradingIndicators.add_rsi(data, price_col="close")
data = TradingIndicators.add_atr(data, high_col="high", low_col="low", close_col="close", window=14, new_col="ATR_14")
data = TradingIndicators.add_high_low_range(data, high_col="high", low_col="low")
data = TradingIndicators.add_buy_pressure(data, high_col="high", low_col="low", close_col="close")
data = TradingIndicators.add_volume_pressure(data, taker_buy_col="taker_buy_quote", total_volume_col="quote_asset_volume")
data = TradingIndicators.add_realized_volatility(data, returns_col="return", window=14)

print(data.head(2))
print(data["target"].value_counts())
print(data.columns)

       open      high       low     close    volume              close_time  \
0  63772.01  63820.00  63757.94  63819.17  62.47638 2024-09-24 08:49:59.999   
1  63819.17  63863.02  63800.00  63861.76  77.04959 2024-09-24 08:54:59.999   

   quote_asset_volume  number_of_trades  taker_buy_quote    return  ...  \
0        3.985620e+06             12768     2.107707e+06       NaN  ...   
1        4.918474e+06             10690     2.507016e+06  0.000667  ...   

   MACD_Signal  Bollinger_SMA  Bollinger_Upper  Bollinger_Lower  RSI_14  \
0     0.000000            NaN              NaN              NaN     NaN   
1     0.679499            NaN              NaN              NaN     NaN   

   ATR_14  High_Low_Range  Buy_Pressure  Volume_Pressure  Realized_Volatility  
0     NaN           62.06      0.986626         0.528828                  NaN  
1     NaN           63.02      0.980006         0.509714                  NaN  

[2 rows x 24 columns]
target
0    88674
1    16447
Name: count, dtype

In [4]:
from models.Models import CryptoModel


crypto_model = CryptoModel(data)


# XGBoost
# xgb_model = crypto_model.xgboost_classification(test_size=0.2)
# Régression logistique
logit_model = crypto_model.logistic_regression(test_size=0.2)

print(logit_model)

KeyboardInterrupt: 